<a href="https://colab.research.google.com/github/Ryan7AIT/datahub/blob/main/rul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=de679d1680be8aa5e0d9b5a5b090b38a6485eaa24ae06df9c87a327921bb7ee1
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
!pip install keras

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, to_date, dayofweek, hour, lag, mean, lit, when,avg
from pyspark.sql.window import Window
import json
from pyspark.sql.functions import rand, isnan, when
from pyspark.sql.types import FloatType
from pyspark.sql.functions import rand, round
from keras.callbacks import TensorBoard
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, monotonically_increasing_id
from pyspark.sql.types import FloatType
import math
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import datetime
from tensorflow.keras.utils import to_categorical

# Initialize a SparkSession
spark = SparkSession.builder.appName("PredictiveMaintenance").getOrCreate()

# Load your dataset into a PySpark DataFrame
df = spark.read.csv('/Users/mac/Downloads/rul4.csv', header=True, inferSchema=True)
df.count()
# df = df.limit(1000)
df.count()
# Assuming 'date_insertion' is the correct timestamp column based on your dataset schema
# Adjust the following transformations accordingly:

# Extract JSON fields function
def extract_from_json(column, key):
    try:
        json_data = json.loads(column.replace("'", "\""))
        return json_data.get(key, None)
    except:
        return None

# Registering the UDF
extract_from_json_udf = udf(extract_from_json)

# Step 1: Extracting 'oil_value' and 'fuel_liters'
df = df.withColumn("oil_value", extract_from_json_udf(col("details"), lit("oil_value")))
df = df.withColumn("fuel_liters", extract_from_json_udf(col("details"), lit("fuel_liters")))

# Step 2: Creating time-based features
# df = df.withColumn("date_insertion", to_date(col("date_insertion")))
df = df.withColumn("day_of_week", dayofweek(col("date_insertion")))
df = df.withColumn("hour_of_day", hour(col("date_insertion")))



# Step 4: Aggregate readings on a daily basis
daily_avg_df = df.groupBy("thing_id", "date_insertion").agg(mean("power_supply_voltage").alias("daily_avg_voltage"))
df = df.join(daily_avg_df, ["thing_id", "date_insertion"], "left")

# Step 5: Create binary indicator for 'engine_status'
df = df.withColumn("engine_alert", when(col("engine_status") == "Abnormal", 1).otherwise(0))

# Define a UDF to generate random values within a range
def random_value(min_value, max_value):
    return (rand() * (max_value - min_value) + min_value).cast(FloatType())

# random_value_udf = udf(random_value, FloatType())

# Set min and max values for 'oil_value' and 'fuel_liters'
oil_value_min, oil_value_max = 0, 4
fuel_liters_min, fuel_liters_max = 0, 60

fuel_liters_mi, fuel_liters_ma = 2, 6


# Replace null values with random numbers
# Replace null values with random numbers and round to 1 decimal place
# df = df.withColumn("oil_value", when(df['oil_value'].isNull(), round((rand() * (oil_value_max - oil_value_min) + oil_value_min), 1)).otherwise(df['oil_value']))


# Sample range for oil values
oil_value_min = 0.5
oil_value_max = 5.0

df = df.orderBy('thing_id', 'date_insertion')

from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

window = Window.partitionBy('thing_id').orderBy('date_insertion')
df = df.withColumn('row_num', row_number().over(window))

from pyspark.sql.functions import count

window = Window.partitionBy('thing_id')
df = df.withColumn('total_rows', count('*').over(window))

from pyspark.sql.functions import when, rand





# Find the maximum total_km value
max_total_km = df.agg(max(col("total_km"))).collect()[0][0]


# Define a window specification to rank rows with the same total_km
window_spec = Window.partitionBy("total_km").orderBy("total_km")

# Add a row number column to the DataFrame
df_with_row_number = df.withColumn("row_number", row_number().over(window_spec))

# Filter rows to keep only one with the maximum total_km value
filtered_max_total_km = df_with_row_number.filter((col("total_km") == max_total_km) & (col("row_number") == 1)).drop("row_number")

# Filter out all rows with the maximum total_km except one
remaining_data = df.filter(col("total_km") != max_total_km)

# Union the remaining data with the single row with the maximum total_km
df = remaining_data.union(filtered_max_total_km)













from pyspark.sql.functions import when, rand, round, col
df = df.withColumn("oil_value", col("oil_value").cast("float"))
# Define the ranges for the oil values
# Define the ranges for the oil values
first_range_min = 4
first_range_max = 6
second_range_min = 3
second_range_max = 6
third_range_min = 2
third_range_max = 5
fourth_range_min = 1.5
fourth_range_max = 3


first_range_min2 = 0.014
first_range_max2 = 0.017
second_range_min2 = 0.014
second_range_max2 = 0.018
third_range_min2 = 0.015
third_range_max2 = 0.018
fourth_range_min2 = 0.017
fourth_range_max2 = 0.022



# Define a window partitioned by 'thing_id' and ordered by 'date_insertion'
window = Window.partitionBy('thing_id').orderBy('date_insertion')
total_window = Window.partitionBy('thing_id')

# Calculate the row number and total rows for each 'thing_id'
df = df.withColumn('row_num', row_number().over(window))
df = df.withColumn('total_rows', count('*').over(total_window))

# Define the thresholds for the different periods
df = df.withColumn('first_threshold', (col('total_rows') * 0.6))
df = df.withColumn('second_threshold', (col('total_rows') * 0.8))
df = df.withColumn('third_threshold', (col('total_rows') * 0.9))

# Generate the oil values
df = df.withColumn('oil_value',
                   when(df['oil_value'].isNull() & (col('row_num') <= col('first_threshold')),
                        round(rand() * (first_range_max - first_range_min) + first_range_min, 1))
                   .when(df['oil_value'].isNull() & (col('row_num') > col('first_threshold')) & (col('row_num') <= col('second_threshold')),
                         round(rand() * (second_range_max - second_range_min) + second_range_min, 1))
                   .when(df['oil_value'].isNull() & (col('row_num') > col('second_threshold')) & (col('row_num') <= col('third_threshold')),
                         round(rand() * (third_range_max - third_range_min) + third_range_min, 1))
                   .when(df['oil_value'].isNull() & (col('row_num') > col('third_threshold')),
                         round(rand() * (fourth_range_max - fourth_range_min) + fourth_range_min, 1))
                   .otherwise(df['oil_value']))


# create an empty fuel change colunm
df = df.withColumn('fuel_change', lit(None).cast(FloatType()))

# generate rhe fule change values
df = df.withColumn('fuel_change',
                   when(df['fuel_change'].isNull() & (col('row_num') <= col('first_threshold')),
                        round(rand() * (first_range_max2 - first_range_min2) + first_range_min2, 3))
                   .when(df['fuel_change'].isNull() & (col('row_num') > col('first_threshold')) & (col('row_num') <= col('second_threshold')),
                         round(rand() * (second_range_max2 - second_range_min2) + second_range_min2, 3))
                   .when(df['fuel_change'].isNull() & (col('row_num') > col('second_threshold')) & (col('row_num') <= col('third_threshold')),
                         round(rand() * (third_range_max2 - third_range_min2) + third_range_min2, 3))
                   .when(df['fuel_change'].isNull() & (col('row_num') > col('third_threshold')),
                         round(rand() * (fourth_range_max2 - fourth_range_min2) + fourth_range_min2, 3))
                   .otherwise(df['fuel_change']))

# Drop the auxiliary columns
df = df.drop('row_num', 'total_rows', 'first_threshold', 'second_threshold', 'third_threshold')


# =====================================
# =====================================
# =====================================
# =====================================
# =====================================
# =====================================
# =====================================
# fuel change


























df = df.withColumn("fuel_liters", when(df['fuel_liters'].isNull(), round((rand() * (fuel_liters_max - fuel_liters_min) + fuel_liters_min), 1)).otherwise(df['fuel_liters']))
# df = df.withColumn("fuel_change", when(df['fuel_liters'].isNull(), round((rand() * (fuel_liters_ma - fuel_liters_mi) + fuel_liters_mi), 1)).otherwise(df['fuel_liters']))








# Step 6: Generate interaction features
# df = df.withColumn("voltage_current_interaction", col("power_supply_voltage") * col("battery_current"))

# Step 3: Calculating rate of change for 'battery_current'
windowSpec = Window.partitionBy("thing_id").orderBy("date_insertion")
df = df.withColumn("battery_current_change", col("power_supply_voltage") - lag("power_supply_voltage", 1).over(windowSpec))


df = df.select("thing_id", "date_insertion", "speed", "total_km", "engine_status", "power_supply_voltage" ,"oil_value","fuel_change", "fuel_liters",  "battery_current_change", "daily_avg_voltage")


from pyspark.sql import Window
from pyspark.sql.functions import lag, avg, stddev

# Define a window
window = Window.orderBy('date_insertion').rowsBetween(-9, 0)  # assuming 'date_insertion' is your time column

# Calculate rolling averages and standard deviations
# df = df.withColumn('speed_avg', avg(df['speed']).over(window))
# df = df.withColumn('oil_value_std', stddev(df['oil_value']).over(window))

# Calculate changes between consecutive readings
# df = df.withColumn('speed_change', df['speed'] - lag(df['speed']).over(Window.orderBy('date_insertion')))
# df = df.withColumn('fuel_change', df['fuel_liters'] - lag(df['fuel_liters']).over(Window.orderBy('date_insertion')))
# df = df.withColumn('fuel_change', (rand() * 4 + 2).cast("int"))#replace all fuel_chnages values with random values between 2 and 6



# Step 7: Calculate rolling average


# Define a Window specification
# windowSpec = Window.orderBy('date_insertion').rowsBetween(-4, 0)  # 5 rows including current row

# Calculate rolling average
# df = df.withColumn('oil_quality_rolling_avg', avg(df['oil_value']).over(windowSpec))

df = df.withColumn("car_age", when(rand() < 0.7, "old").otherwise("new"))

# Show the first 5 rows of the DataFrame






/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/mac/Downloads/rul4.csv.